In [2]:
import sys
!conda install --yes --prefix {sys.prefix} sqlalchemy

# this is how you can conda install a package with in Jupyter Notebook

#For pip ->  


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment C:\miniconda3\envs\data:

The following NEW packages will be INSTALLED:

    sqlalchemy: 1.2.7-py36ha85dd04_0

sqlalchemy-1.2   0% |                              | ETA:  --:--:--   0.00  B/s
sqlalchemy-1.2   0% |                               | ETA:  0:04:37   5.90 kB/s
sqlalchemy-1.2   2% |                               | ETA:  0:01:46  15.08 kB/s
sqlalchemy-1.2   3% |#                              | ETA:  0:04:00   6.61 kB/s
sqlalchemy-1.2   4% |#                              | ETA:  0:04:02   6.48 kB/s
sqlalchemy-1.2   5% |#                              | ETA:  0:03:50   6.77 kB/s
sqlalchemy-1.2   7% |##                             | ETA:  0:03:02   8.34 kB/s
sqlalchemy-1.2   8% |##                             | ETA:  0:02:54   8.63 kB/s
sqlalchemy-1.2 100% |###############################| Time: 0:01:17  21.27 kB/s

sqlalchemy-1.2   0% |                           

In [5]:
!{sys.executable} -m pip install sqlalchemy #install package through pip in Jupyter Notebook

In [7]:
!conda install --yes --prefix {sys.prefix} psycopg2

#psycopg2 is needed to work with SQLAlchemy and Postgres. It provides Python interface to talk to Database 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment C:\miniconda3\envs\data:

The following NEW packages will be INSTALLED:

    krb5:     1.16-he625fcf_6      
    libpq:    10.3-hc4dcbb0_0      
    psycopg2: 2.7.4-py36h74b6da3_0 

The following packages will be UPDATED:

    openssl:  1.0.2l-vc14hb3c0c5f_5 --> 1.0.2o-h8ea7d77_0

openssl-1.0.2o   0% |                              | ETA:  --:--:--   0.00  B/s
openssl-1.0.2o   1% |                               | ETA:  0:00:38 112.53 kB/s
openssl-1.0.2o   2% |                               | ETA:  0:00:30 140.03 kB/s
openssl-1.0.2o   3% |#                              | ETA:  0:00:22 192.90 kB/s
openssl-1.0.2o   4% |#                              | ETA:  0:00:19 210.05 kB/s
openssl-1.0.2o   5% |#                              | ETA:  0:00:20 204.60 kB/s
openssl-1.0.2o   6% |##                             | ETA:  0:00:19 205.48 kB/s
openssl-1.0.2o   7% |##               

In [7]:
from sqlalchemy import create_engine, Column, String, Integer, Text, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

engine = create_engine('sqlite:///:memory:', echo = False)
Base = declarative_base()

In [8]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    #addresses = relationship("Address", order_by=Address.id, back_populates="user")
    def __repr__(self):
        return "<User(name = '%s', fullname = '%s', password = '%s')>" %(self.name, self.fullname, self.password)
    
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key = True)
    email_address = Column(String, nullable = False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates ='addresses')
    # Address.user will return User instances associated with a email-address
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

User.addresses = relationship("Address", order_by=Address.id, back_populates="user") 
#-> User.addresses will return email-addresses associated with a given User instance.
    

In [23]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind = engine)
session = Session()


In [24]:
session.add_all([
User(name = 'sam', fullname = 'sam james', password = 'x'),
User(name = 'pete', fullname = 'pete james', password = 'y'),
User(name = 'max', fullname = 'max mac', password = 'y')])
session.new
session.commit()
a = session.query(User)
for i in a:
    print(i)

session.add_all([
Address(email_address = 'a@gmail.com', user_id = 1),
Address(email_address = 'b@gmail.com', user_id = 1),
Address(email_address = 'c@gmail.com', user_id = 2)])
session.new
session.commit()

    

<User(name = 'sam', fullname = 'sam james', password = 'x')>
<User(name = 'pete', fullname = 'pete james', password = 'y')>
<User(name = 'max', fullname = 'max mac', password = 'y')>
<User(name = 'sam', fullname = 'sam james', password = 'x')>
<User(name = 'pete', fullname = 'pete james', password = 'y')>
<User(name = 'max', fullname = 'max mac', password = 'y')>
<User(name = 'sam', fullname = 'sam james', password = 'x')>
<User(name = 'pete', fullname = 'pete james', password = 'y')>
<User(name = 'max', fullname = 'max mac', password = 'y')>
<User(name = 'sam', fullname = 'sam james', password = 'x')>
<User(name = 'pete', fullname = 'pete james', password = 'y')>
<User(name = 'max', fullname = 'max mac', password = 'y')>


In [19]:
user1 = list(a)[0]
user1

<User(name = 'sam', fullname = 'sam james', password = 'x')>

In [26]:
user1.addresses

[<Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>]

In [25]:
b = session.query(Address)
list(b)

[<Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='c@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='c@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='c@gmail.com')>,
 <Address(email_address='a@gmail.com')>,
 <Address(email_address='b@gmail.com')>,
 <Address(email_address='c@gmail.com')>]

In [27]:
list(b)[0].user

<User(name = 'sam', fullname = 'sam james', password = 'x')>

In [28]:
list(b)[1].user

<User(name = 'sam', fullname = 'sam james', password = 'x')>

In [29]:
list(b)[2].user

<User(name = 'pete', fullname = 'pete james', password = 'y')>